Making sure that the angle array matches correctly:

In [1]:
import numpy as np

In [2]:
np.arange(0, 361, 1.)

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142.,
       143., 144., 145., 146., 147., 148., 149., 15

In [4]:
(np.arange(0, 361, 1.) * np.pi / 180.)

array([0.        , 0.01745329, 0.03490659, 0.05235988, 0.06981317,
       0.08726646, 0.10471976, 0.12217305, 0.13962634, 0.15707963,
       0.17453293, 0.19198622, 0.20943951, 0.2268928 , 0.2443461 ,
       0.26179939, 0.27925268, 0.29670597, 0.31415927, 0.33161256,
       0.34906585, 0.36651914, 0.38397244, 0.40142573, 0.41887902,
       0.43633231, 0.45378561, 0.4712389 , 0.48869219, 0.50614548,
       0.52359878, 0.54105207, 0.55850536, 0.57595865, 0.59341195,
       0.61086524, 0.62831853, 0.64577182, 0.66322512, 0.68067841,
       0.6981317 , 0.71558499, 0.73303829, 0.75049158, 0.76794487,
       0.78539816, 0.80285146, 0.82030475, 0.83775804, 0.85521133,
       0.87266463, 0.89011792, 0.90757121, 0.9250245 , 0.9424778 ,
       0.95993109, 0.97738438, 0.99483767, 1.01229097, 1.02974426,
       1.04719755, 1.06465084, 1.08210414, 1.09955743, 1.11701072,
       1.13446401, 1.15191731, 1.1693706 , 1.18682389, 1.20427718,
       1.22173048, 1.23918377, 1.25663706, 1.27409035, 1.29154

Now, we check the propagators:

In [8]:
def calculate_k_dot_delta(
    squared_Q_momentum_transfer: float, 
    x_Bjorken: float, 
    squared_hadronic_momentum_transfer_t: float,
    azimuthal_phi: float,
    epsilon: float, 
    lepton_energy_fraction_y: float,
    kinematic_k: float,
    verbose: bool = False):
    """
    Description
    --------------
    Equation (29) in the BKM Formalism, available
    at this link: https://arxiv.org/pdf/hep-ph/0112108.pdf

    Parameters
    --------------
    kinematic_k: (float)
    
    epsilon: (float)

    squared_Q_momentum_transfer: (float)

    x_Bjorken: (float)

    lepton_energy_fraction_y: (float)

    squared_hadronic_momentum_transfer_t: (float)

    azimuthal_phi: (float)

    verbose: (bool)
        Debugging console output.

    Returns
    --------------
    k_dot_delta_result : (float)
        result of the operation
    
    Notes
    --------------
    (1): k-dot-delta shows up in computing the lepton
        propagators. It is Eq. (29) in the following
        paper: https://arxiv.org/pdf/hep-ph/0112108.pdf
    """

    try:
    
        # (1): The prefactor: \frac{Q^{2}}{2 y (1 + \varepsilon^{2})}
        prefactor = squared_Q_momentum_transfer / (2. * lepton_energy_fraction_y * (1. + epsilon**2))

        # (2): Second term in parentheses: Phi-Dependent Term: 2 K np.cos(\phi)
        phi_dependence = 2. * kinematic_k * np.cos(np.pi - azimuthal_phi)
        
        # (3): Prefactor of third term in parentheses: \frac{t}{Q^{2}}
        ratio_delta_to_q_squared = squared_hadronic_momentum_transfer_t / squared_Q_momentum_transfer

        # (4): Second term in the third term's parentheses: x_{B} (2 - y)
        bjorken_scaling = x_Bjorken * (2. - lepton_energy_fraction_y)

        # (5): Third term in the third term's parentheses: \frac{y \varepsilon^{2}}{2}
        ratio_y_epsilon = lepton_energy_fraction_y * epsilon**2 / 2.

        # (6): Adding up all the "correction" pieces to the prefactor, written as (1 + correction)
        correction = phi_dependence - (ratio_delta_to_q_squared * (1. - bjorken_scaling + ratio_y_epsilon)) + (ratio_y_epsilon)

        # (7): Writing it explicitly as "1 + correction"
        in_parentheses = 1. + correction

        # (8): The actual equation:
        k_dot_delta_result = -1. * prefactor * in_parentheses

        # (8.1): If verbose, print the output:
        if verbose:
            print(f"> Calculated k dot delta: {k_dot_delta_result}")

        # (9): Return the number:
        return k_dot_delta_result
    
    except Exception as E:
        print(f"> Error in calculating k.Delta:\n> {E}")
        return 0.

In [10]:
k_dot_delta = calculate_k_dot_delta(
    squared_Q_momentum_transfer=1.82,
    x_Bjorken=0.34,
    squared_hadronic_momentum_transfer_t=-0.17,
    azimuthal_phi=(np.arange(0, 361, 1.) * np.pi / 180.),
    epsilon=0.4729356100497334,
    lepton_energy_fraction_y=0.4960961235592845,
    kinematic_k=0.08492693191323897,
    verbose=True)

> Calculated k dot delta: [-1.4037779  -1.40381668 -1.40393301 -1.40412684 -1.40439813 -1.4047468
 -1.40517272 -1.40567578 -1.40625582 -1.40691266 -1.4076461  -1.40845593
 -1.40934189 -1.41030372 -1.41134111 -1.41245376 -1.41364133 -1.41490345
 -1.41623974 -1.4176498  -1.41913318 -1.42068945 -1.42231813 -1.42401871
 -1.42579069 -1.42763353 -1.42954665 -1.43152949 -1.43358143 -1.43570185
 -1.43789011 -1.44014553 -1.44246743 -1.44485511 -1.44730784 -1.44982486
 -1.45240542 -1.45504872 -1.45775396 -1.46052032 -1.46334696 -1.46623301
 -1.46917759 -1.47217981 -1.47523875 -1.47835349 -1.48152306 -1.48474652
 -1.48802286 -1.49135111 -1.49473024 -1.49815922 -1.50163701 -1.50516255
 -1.50873477 -1.51235258 -1.51601487 -1.51972053 -1.52346844 -1.52725744
 -1.53108639 -1.53495412 -1.53885945 -1.54280119 -1.54677814 -1.55078909
 -1.55483282 -1.5589081  -1.56301368 -1.56714831 -1.57131074 -1.5754997
 -1.5797139  -1.58395208 -1.58821293 -1.59249515 -1.59679745 -1.60111852
 -1.60545703 -1.60981167 -1

In [12]:
p1_propagator = 1. + (2. * (k_dot_delta / 1.82))
p1_propagator

array([-0.54261308, -0.54265569, -0.54278352, -0.54299653, -0.54329465,
       -0.5436778 , -0.54414585, -0.54469866, -0.54533606, -0.54605787,
       -0.54686385, -0.54775377, -0.54872735, -0.5497843 , -0.5509243 ,
       -0.55214699, -0.55345201, -0.55483896, -0.55630741, -0.55785692,
       -0.55948701, -0.5611972 , -0.56298695, -0.56485573, -0.56680296,
       -0.56882805, -0.57093039, -0.57310933, -0.57536421, -0.57769434,
       -0.58009902, -0.58257751, -0.58512905, -0.58775287, -0.59044817,
       -0.59321413, -0.59604991, -0.59895464, -0.60192743, -0.60496739,
       -0.60807358, -0.61124506, -0.61448087, -0.61778001, -0.62114149,
       -0.62456427, -0.62804732, -0.63158958, -0.63518996, -0.63884737,
       -0.6425607 , -0.64632881, -0.65015056, -0.65402479, -0.6579503 ,
       -0.66192591, -0.66595041, -0.67002256, -0.67414114, -0.67830488,
       -0.68251251, -0.68676277, -0.69105434, -0.69538592, -0.6997562 ,
       -0.70416384, -0.7086075 , -0.71308582, -0.71759745, -0.72

In [14]:
p2_propagator = (-2. * (k_dot_delta / 1.82)) + (-0.17 / 1.82)
p2_propagator

array([1.44920648, 1.4492491 , 1.44937693, 1.44958994, 1.44988806,
       1.4502712 , 1.45073925, 1.45129206, 1.45192947, 1.45265127,
       1.45345726, 1.45434718, 1.45532076, 1.45637771, 1.45751771,
       1.4587404 , 1.46004542, 1.46143236, 1.46290082, 1.46445033,
       1.46608042, 1.46779061, 1.46958036, 1.47144914, 1.47339637,
       1.47542146, 1.4775238 , 1.47970274, 1.48195762, 1.48428775,
       1.48669243, 1.48917091, 1.49172246, 1.49434628, 1.49704158,
       1.49980754, 1.50264331, 1.50554804, 1.50852084, 1.51156079,
       1.51466699, 1.51783847, 1.52107427, 1.52437342, 1.52773489,
       1.53115768, 1.53464073, 1.53818298, 1.54178337, 1.54544078,
       1.54915411, 1.55292222, 1.55674397, 1.56061819, 1.56454371,
       1.56851932, 1.57254381, 1.57661597, 1.58073454, 1.58489828,
       1.58910592, 1.59335617, 1.59764774, 1.60197933, 1.6063496 ,
       1.61075724, 1.6152009 , 1.61967923, 1.62419085, 1.62873441,
       1.63330851, 1.63791175, 1.64254275, 1.64720008, 1.65188